In [1]:
import os, sys, time, re, json
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
from io import BytesIO
import pickle
import random
from tqdm import tqdm

In [2]:
from unrealcv import client
client.connect()
if not client.isconnected():
    print('UnrealCV server is not running. Run the game downloaded from http://unrealcv.github.io first.')
    sys.exit(-1)
print(client.request('vget /unrealcv/status'))

INFO:__init__:234:Got connection confirm: b'connected to CityPark'


=> Info: Use inet client...
=>Info: using ip-port socket
Is Listening
Client Connected
9000
Configuration
Config file: C:/Program Files/Epic Games/UE_4.27/Engine/Binaries/Win64/unrealcv.ini
Port: 9000
Width: 640
Height: 480
FOV: 90.000000
EnableInput: true
EnableRightEye: false



In [3]:
def parse_color(color_str):
    # Use regular expression to find all numbers
    numbers = re.findall(r'\d+', color_str)

    # Convert the list of strings to a list of integers
    array = list(map(int, numbers))
    return (array[0], array[1], array[2])

def parse_rotation(rotation_str):
    # Use regular expression to find all numbers including negatives and decimals
    numbers = re.findall(r'-?\d+\.\d+', rotation_str)

    # Convert the list of strings to a list of floats
    array = list(map(float, numbers))
    return (array[0], array[1], array[2])

In [4]:
# Get the list of all objects in the scene
scene_objects = client.request('vget /objects').split(' ')
scene_objects.pop()

person_colors = set()
for obj in scene_objects:
    color_str = client.request('vget /object/%s/color' % obj)
    color = parse_color(color_str)
    if (obj.startswith('SM_Child') or obj.startswith('SM_Couple') or obj.startswith('SM_Man') or obj.startswith('SM_Trio') or obj.startswith('SM_Woman')) and not obj.startswith('SM_Manhole'):
        if not color in person_colors:
            person_colors.add(color)
        else:
            print('Duplicate color:', color)
            sys.exit(-1)
    elif color in person_colors:
        print('Duplicate color:', color)
        sys.exit(-1)

with open('./data/my_set.pkl', 'wb') as file:
    pickle.dump(person_colors, file)

In [103]:
IMG_WIDTH = 1080
IMG_HEIGHT = 720
IMG_FOV = 80

CAMERAS = len(client.request('vget /cameras').split(' ')) - 1

for i in range(CAMERAS):
    client.request('vset /camera/%d/size %d %d' % (i, IMG_WIDTH, IMG_HEIGHT))
    client.request('vset /camera/%d/fov %d' % (i, IMG_FOV))

In [ ]:
rotation_factor = 400

In [ ]:
i = 0
for cam in range(1, CAMERAS): # skip the first camera
    # get camera rotation
    rot_str = client.request('vget /camera/%d/rotation' % cam)
    rot = parse_rotation(rot_str)
    rotation_angle = 0
    while rotation_angle < 360:
        # set camera rotation
        client.request('vset /camera/%d/rotation %f %f %f' % (cam, rot[0], rotation_angle, rot[2]))
        rotation_angle += rotation_factor
        i += 1
        # get the object mask
        object_mask_bytes = client.request('vget /camera/%d/object_mask png' % cam)
        object_mask = PIL.Image.open(BytesIO(object_mask_bytes))
        object_mask.save("./data/seg/%d.png" % i)

In [82]:
i = 0
for cam in range(1, CAMERAS): # skip the first camera
    # get camera rotation
    rot_str = client.request('vget /camera/%d/rotation' % cam)
    rot = parse_rotation(rot_str)
    rotation_angle = 0
    while rotation_angle < 360:
        # set camera rotation
        client.request('vset /camera/%d/rotation %f %f %f' % (cam, rot[0], rotation_angle, rot[2]))
        rotation_angle += rotation_factor
        i += 1
        # get the object mask
        object_mask_bytes = client.request('vget /camera/%d/lit png' % cam)
        object_mask = PIL.Image.open(BytesIO(object_mask_bytes))
        object_mask.save("./data/seg/%d_1.png" % i)


(-21.08, -95.091, -0.0)
(-18.6, 87.509, 0.0)
(-18.8, -179.891, 0.0)
(-14.6, -38.0, 0.0)
(-22.2, -169.8, 0.0)
(-17.199, 58.0, 0.0)
(-16.6, 117.0, 0.0)
(-19.799, 83.2, 0.0)
(-12.2, -83.8, -0.0)
(-43.799, 12.4, 0.0)
(-22.0, -102.2, 0.0)
(-23.8, -123.202, 0.0)
(-15.2, 106.199, 0.0)
(-25.4, -86.8, 0.0)
(-21.4, -152.6, 0.0)
(-19.601, -111.196, 0.0)
(-12.601, -179.395, 0.0)
(-23.001, -160.793, 0.0)


In [1]:
# Find max_x, max_y, min_x, min_y pixels of a color in an object mask
def get_color_box(color, object_mask):
    max_x = 0
    max_y = 0
    min_x = object_mask.shape[1]
    min_y = object_mask.shape[0]
    is_present = False
    for y in range(object_mask.shape[0]):
        for x in range(object_mask.shape[1]):
            if np.array_equal(object_mask[y][x], color):
                is_present = True
                if x > max_x:
                    max_x = x
                if y > max_y:
                    max_y = y
                if x < min_x:
                    min_x = x
                if y < min_y:
                    min_y = y
    return (is_present, min_x, min_y, max_x, max_y)


def bounding_boxes(object_mask, color_set):
    bounding_boxes = []
    for color in color_set:
        is_present, min_x, min_y, max_x, max_y = get_color_box(color, object_mask)
        if is_present:
            # COCO format: [x_min, y_min, width, height]
            width = max_x - min_x
            height = max_y - min_y
            if width < IMG_WIDTH/80 or height < IMG_HEIGHT/80:
                continue
            bounding_boxes.append([min_x, min_y, width, height])
    return bounding_boxes


def create_coco_format_json(filepaths, color_set):

    images = []
    annotations = []
    categories = [{ "id": 0, "name": "person"}]
    count = 0
    
    # Iterate over image filepaths
    for filepath in tqdm(filepaths):
        # Get the file name
        file_name = filepath.split("/")[-1]
        # Get the image id
        file_id = file_name.split("_")[0]

        # Open the segmentation image
        img = PIL.Image.open("./data/seg/" + file_name)
        width, height = img.size
        
        
        # Adding images which has annotations
        images.append(
            {
                "id": file_id,
                "width": width,
                "height": height,
                "file_name": file_name
            }
        )

        bounding_boxes = bounding_boxes(img, color_set)

        for box in bounding_boxes:
            seg = {
                'bbox': box,
                'image_id': file_id, 
                'category_id': 0, 
                'iscrowd': 0, 
                'id': count
            }
            annotations.append(seg)
            count +=1
            
    # Create the dataset
    dataset_coco_format = {
        "categories": categories,
        "images": images,
        "annotations": annotations,
    }
    
    return dataset_coco_format